In [1]:
#imports
import dask
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.array as da
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from dask_ml import cluster, decomposition, linear_model, metrics, xgboost
from dask_ml.impute import SimpleImputer
from dask_ml.model_selection import train_test_split, KFold
from dask_ml.preprocessing import OneHotEncoder 
from zipfile import ZipFile 
import pandas as pd
import numpy as np

def cat_features(dataframe):
    td = pd.DataFrame({'a': [1,2,3], 'b': [1.0, 2.0, 3.0]})
    return filter(lambda x: not(dataframe[x].dtype in [td['a'].dtype, td['b'].dtype]), list(dataframe))

#close existing dask connection if it exists and open a new one
try:
    if client is not None:
        client.close()
        print("closed existing connection, ",client)       
except Exception as e:
    print(e)

name 'client' is not defined


In [2]:
#get new connection
client = Client(n_workers=2, threads_per_worker=2, memory_limit='8GB')
display(client)

Client Scheduler: tcp://127.0.0.1:43831 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 16.00 GB


In [3]:
# Declare the dtypes of the file to be more efficient when reading in 
feat_types = {'Dst Port': np.float64, 'Protocol': np.int64, 'Timestamp': str, 
              'Flow Duration': np.int64, 'Tot Fwd Pkts': np.int64, 'Tot Bwd Pkts': np.int64, 
              'TotLen Fwd Pkts': np.int64, 'TotLen Bwd Pkts': np.int64, 'Fwd Pkt Len Max': np.int64, 
              'Fwd Pkt Len Min': np.int64, 'Fwd Pkt Len Mean': np.float64, 'Fwd Pkt Len Std': np.float64, 
              'Bwd Pkt Len Max': np.int64, 'Bwd Pkt Len Min': np.int64, 'Bwd Pkt Len Mean': np.float64, 
              'Bwd Pkt Len Std': str, 'Flow Byts/s': str, 'Flow Pkts/s': np.float64, 
              'Flow IAT Mean': np.float64, 'Flow IAT Std': np.float64, 'Flow IAT Max': np.int64, 
              'Flow IAT Min': np.int64, 'Fwd IAT Tot': np.int64, 'Fwd IAT Mean': np.float64, 
              'Fwd IAT Std': np.float64, 'Fwd IAT Max': np.int64, 'Fwd IAT Min': np.int64, 
              'Bwd IAT Tot': np.int64, 'Bwd IAT Mean': np.float64, 'Bwd IAT Std': np.float64, 
              'Bwd IAT Max': np.int64, 'Bwd IAT Min': np.int64, 'Fwd PSH Flags': np.int64, 
              'Bwd PSH Flags': np.int64, 'Fwd URG Flags': np.int64, 'Bwd URG Flags': np.int64, 
              'Fwd Header Len': np.int64, 'Bwd Header Len': np.int64, 'Fwd Pkts/s': np.float64, 
              'Bwd Pkts/s': np.float64, 'Pkt Len Min': np.int64, 'Pkt Len Max': np.int64, 
              'Pkt Len Mean': np.float64, 'Pkt Len Std': np.float64, 'Pkt Len Var': np.float64, 
              'FIN Flag Cnt': np.int64, 'SYN Flag Cnt': np.int64, 'RST Flag Cnt': np.int64, 
              'PSH Flag Cnt': np.int64, 'ACK Flag Cnt': np.int64, 'URG Flag Cnt': np.int64, 
              'CWE Flag Count': np.int64, 'ECE Flag Cnt': np.int64, 'Down/Up Ratio': np.int64, 
              'Pkt Size Avg': np.float64, 'Fwd Seg Size Avg': np.float64, 'Bwd Seg Size Avg': np.float64, 
              'Fwd Byts/b Avg': np.int64, 'Fwd Pkts/b Avg': np.int64, 'Fwd Blk Rate Avg': np.int64, 
              'Bwd Byts/b Avg': np.int64, 'Bwd Pkts/b Avg': np.int64, 'Bwd Blk Rate Avg': np.int64, 
              'Subflow Fwd Pkts': np.int64, 'Subflow Fwd Byts': np.int64, 'Subflow Bwd Pkts': np.int64, 
              'Subflow Bwd Byts': np.int64, 'Init Fwd Win Byts': np.int64, 'Init Bwd Win Byts': np.int64, 
              'Fwd Act Data Pkts': np.int64, 'Fwd Seg Size Min': np.int64, 'Active Mean': np.float64, 
              'Active Std': np.float64, 'Active Max': np.int64, 'Active Min': np.int64, 
              'Idle Mean': np.float64, 'Idle Std': np.float64, 'Idle Max': np.int64,
              'Idle Min': np.int64, 'Label': str}

feat_types_check = {'Dst Port': float, 'Protocol': int, 'Timestamp': str, 
              'Flow Duration': int, 'Tot Fwd Pkts': int, 'Tot Bwd Pkts': int, 
              'TotLen Fwd Pkts': int, 'TotLen Bwd Pkts': int, 
              'Flow Pkts/s': float,  'Fwd PSH Flags': int, 
              'Bwd PSH Flags': int, 'Fwd URG Flags': int, 'Bwd URG Flags': int, 
              'Fwd Pkts/s': float, 
              'Bwd Pkts/s': float,
              'FIN Flag Cnt': int, 'SYN Flag Cnt': int, 'RST Flag Cnt': int, 
              'PSH Flag Cnt': int, 'ACK Flag Cnt': int, 'URG Flag Cnt': int, 
              'CWE Flag Count': int, 'ECE Flag Cnt': int, 
              'Subflow Fwd Pkts': int, 'Subflow Fwd Byts': int, 'Subflow Bwd Pkts': int, 
              'Subflow Bwd Byts': int, 'Label': str}

feature_cols = ['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 
                'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
                'TotLen Bwd Pkts', 'Flow Pkts/s', 
                'Flow IAT Mean', 'Fwd PSH Flags', 'Bwd PSH Flags', 
                'Fwd URG Flags','Bwd URG Flags', 'FIN Flag Cnt', 
                'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 
                'ACK Flag Cnt', 'URG Flag Cnt','CWE Flag Count', 
                'ECE Flag Cnt', 'Subflow Fwd Pkts','Subflow Fwd Byts', 
                'Subflow Bwd Pkts', 'Subflow Bwd Byts', 'Label']
# print(len(feat_types_check))
# print(feature_cols[18])
df = dd.read_csv('cic_data/23_02_2018.csv', usecols=feature_cols, low_memory=False)
df_2 = dd.read_csv('cic_data/16_02_2018_fixeddata.csv', usecols=feature_cols, low_memory=False)
df = df.append(df_2)
print("Length of Dataset: ", len(df))
print("Number of Columns: ", len(df.columns))
# print("Categorical Features: ", list(cat_features(df)))
# Files in the zipped folder:
# 23_02_2018.csv, 20_02_2018.csv, 16_02_2018.csv, 02_03_2018.csv
# with ZipFile('cic_data.zip') as zipped:
#     df = dd.from_pandas(pd.read_csv(zipped.open('23_02_2018.csv'), usecols=feature_cols), npartitions=2)
#     df = df.append(pd.read_csv(zipped.open('16_02_2018.csv'), usecols=feature_cols))

Length of Dataset:  2097149
Number of Columns:  27


In [4]:
x_cols = ['Dst Port', 'Protocol', 'Flow Duration', 
                'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
                'TotLen Bwd Pkts', 'Flow Byts/s', 'Flow Pkts/s', 
                'Flow IAT Mean', 'Fwd PSH Flags', 'Bwd PSH Flags', 
                'Fwd URG Flags','Bwd URG Flags', 'FIN Flag Cnt', 
                'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 
                'ACK Flag Cnt', 'URG Flag Cnt','CWE Flag Count', 
                'ECE Flag Cnt', 'Subflow Fwd Pkts','Subflow Fwd Byts', 
                'Subflow Bwd Pkts', 'Subflow Bwd Byts']
df['Target'] = 0 
df['Target'] = df['Target'].mask(df['Label'] != 'Benign', 1)

In [5]:
for col in df.columns:
    if col in feat_types_check.keys():
        df[col] = df[col].astype(feat_types_check[col])

In [5]:
y = df['Target']
# drop_cols = [col for col in df.columns if col not in x_cols]
x = df.drop(['Label', 'Target', 'Timestamp'], axis=1)

In [6]:
#function that takes X, Y and gives you a train test split
def doTrainTestSplit(X,Y):
    #breakpoint()
    #look at dask dataframes
#     display(X.head())
#     display(Y.head())
   
    #get splits
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=.3)
#     display(X_train.compute())
#     display(y_train.compute())
    return X_train, X_test, y_train, y_test 

In [7]:
x_train, x_test, y_train, y_test = doTrainTestSplit(x,y)
x_train = x_train.to_dask_array()

In [8]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [9]:
x_train = imp.fit_transform(x_train)
# x_test = imp.transform(x_test)

In [ ]:
xgb_clf = xgboost.XGBClassifier(n_estimators=5)
xgb_clf.fit(x_train, y_train)

distributed.nanny - WARNING - Worker process 7559 was killed by signal 6
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process 7557 was killed by signal 6
distributed.nanny - WARNING - Restarting worker
